## NoteBook for the first project
- Je pensais que l'on pouvait essayer de faire ca comme ceci:
    - Chacun de nous a un block (peut bien entendu en ajouter en dessous du sien) afin de ne pas avoir trop de conflit lorsque l'on git push
        - A tester cette semaine pour voir si c'est ok

# Stefan's Blocks

In [ ]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from helpers import *
from proj1_helpers import * 
from play_with_data import * # Contains some functions that allow us to look the data and manipulate a little bit
from pre_processing import * # Contains all the functions needed to do some pre-processing on the data
%load_ext autoreload
%autoreload 2



In [ ]:
# ********** Import csv file ***********

# Import of train.csv --> This was my way of doing it
# id_train, labels_train, features_train = load_data("train.csv")

# By using the helpers 
data_path = "train.csv"
labels_train, features_train, id_train = load_csv_data(data_path)

# Import of test.csv --> This is my way of doing it, because it is not done in proj1_helpers
id_test, features_test = load_data("test.csv")

In [ ]:
# ********* Data whitening and "build model" ***********
# I think the order of the two functions below is important, because if we apply the whitening on the column of 1 it doesn't make sense I thin, do you???????

features_train = data_whitening(features_train) # to whiten the data
# Create the data for the model --> add a column of "1" as the first column --> this is for the w0 (offset parameter)
features_train = build_model_data(features_train, labels_train)[1] # the "[1]" is to take only the "X" matrix and not the "y"

In [ ]:
features_bad_ind = np.unique(np.where(features_train == -999.0)[1])
features_train = np.delete(features_train,features_bad_ind,1)
print(features_train.shape)

In [ ]:
for i in range(features_train.shape[1]):
    for j in list(range(i + 1)):    
        newFeature = np.multiply(features_train[:,i],features_train[:,j])
        features_train = np.c_[features_train,newFeature]


In [ ]:
print(a.shape)

#### Do a cross validation with logistic_regression on the data without the features with -999.0

#### Hyperparameters:
- Gamma
- K --> number of sets for the cross validation 

In [ ]:
from cross_val import *

# Prepare the data, remove the features with at least one value at -999.0
data_features_removed = remove_features(features_train)

seed = 1
degree = 5 # No more useful
k_fold = 15 # I have seen a decreasing in the error when increasing the number of fold
gammas = np.linspace(-0.5,0.5,10)
#lambdas = np.logspace(-4, 7, 40)# No longer useful because we don't use anymore the ridge regression


# split data in k fold
k_indices = build_k_indices(labels_train, k_fold, seed)
# define lists to store the loss of training data and test data
allValidationError = []

for g in gammas: # To see with which gamma we obtained a better result
    validation_error = 0
    for k in range(k_fold):
        validation_error = validation_error + cross_validation(labels_train, data_features_removed, k_indices, k, g) 
    allValidationError.append(validation_error/k_fold) # we compute the mean of the validation error
    


In [ ]:
# In order to have some information about the cross validation we just did
plt.close("all")
plt.plot(gammas, allValidationError )
print(allValidationError)
print(min(allValidationError))
plt.show()

# FUNCTIONS --> but not so sure of them! Maybe need to re-do (I can put the one I've did from the exercise, I'll do this Friday afternon)

In [ ]:
# To compute the losse, with mse --> with solution
def compute_mse(y, tx, w):
    """Calculate the loss.

    You can calculate the loss using mse or mae.
    """
    """compute the loss by mse."""
    e = y - tx.dot(w)
    mse = e.dot(e) / (2 * len(e)) # Here if we have any problem we should try with a transpose
    return mse

In [ ]:
# Gradient descent

def compute_gradient(y, tx, w):
    """Compute the gradient."""

    # compute gradient and loss
    e = y - np.dot(tx, w)
    gradient = -1/len(y) * np.dot(tx.T, e)
    
    loss = compute_mse(y,tx,w)
    return gradient, loss


def gradient_descent(y, tx, initial_w, max_iters, gamma):
    """Gradient descent algorithm."""
    # Define parameters to store w and loss
    ws = [initial_w]
    losses = []
    w = initial_w 
    for n_iter in range(max_iters):
        # compute gradient and loss
        gradient, loss = compute_gradient(y,tx,w)

        # update w by gradient
        w = ws[n_iter] - gamma*gradient
        
        # store w and loss
        ws.append(w)
        losses.append(loss)
        
        print("Gradient Descent({bi}/{ti}): loss={l}, w0={w0}, w1={w1}".format(
              bi=n_iter, ti=max_iters - 1, l=loss, w0=w[0], w1=w[1]))

    return losses[-1], ws[-1,:]

In [ ]:
# Stochastic gradient descent
def batch_iter(y, tx, batch_size, num_batches=1, shuffle=True):
    """
    Generate a minibatch iterator for a dataset.
    Takes as input two iterables (here the output desired values 'y' and the input data 'tx')
    Outputs an iterator which gives mini-batches of `batch_size` matching elements from `y` and `tx`.
    Data can be randomly shuffled to avoid ordering in the original data messing with the randomness of the minibatches.
    Example of use :
    for minibatch_y, minibatch_tx in batch_iter(y, tx, 32):
        <DO-SOMETHING>
    """
    data_size = len(y)

    if shuffle:
        shuffle_indices = np.random.permutation(np.arange(data_size))
        shuffled_y = y[shuffle_indices]
        shuffled_tx = tx[shuffle_indices]
    else:
        shuffled_y = y
        shuffled_tx = tx
    for batch_num in range(num_batches):
        start_index = batch_num * batch_size
        end_index = min((batch_num + 1) * batch_size, data_size)
        if start_index != end_index:
            yield shuffled_y[start_index:end_index], shuffled_tx[start_index:end_index]
            

def compute_stoch_gradient(y, tx, w):
    """Compute a stochastic gradient from just few examples n and their corresponding y_n labels."""
    e = y - np.dot(tx, w)
    gradient = -1/len(y) * np.dot(tx.T, e)
    
    loss = compute_loss(y,tx,w)
    return gradient, loss


def stochastic_gradient_descent(y, tx, initial_w, max_iters, gamma, batch_size=1):
    """Stochastic gradient descent algorithm."""
    ws = [initial_w]
    losses = []
    lossForOneRun = 0
    
    for n_iters in range(max_iters):
        for minibatch_y, minibatch_tx in batch_iter(y, tx, batch_size):
            gradient, loss = compute_stoch_gradient(minibatch_y, minibatch_tx, ws[n_iters])
            lossForOneRun = lossForOneRun + loss       #Faudrait pas reinitialiser avant chaque iteration
            w = ws[n_iters] - gamma*gradient
        
        losses.append((1/batch_size)*lossForOneRun)
        ws.append(w)
    
    return losses[-1], ws[-1,:]

In [ ]:
# Least squares --> With solution
def least_squares(y, tx):
    """calculate the least squares solution."""
    
    a = tx.T.dot(tx)
    b = tx.T.dot(y)
    w = np.linalg.solve(a, b)
    loss = compute_mse(y, tx, w)
    
    return w, loss


In [ ]:
# Ridge_regression using normal equations --> with solution
def ridge_regression(y, tx, lambda_):
    """implement ridge regression."""

    aI = 2 * tx.shape[0] * lambda_*np.identity(tx.shape[1])
    a = tx.T.dot(tx) + aI
    b = tx.T.dot(y)
    
    w = np.linalg.solve(a,b)
    loss = compute_mse(y, tx, w)
    
    return w, loss

# Etienne's Blocks

In [ ]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from helpers import *
%load_ext autoreload
%autoreload 2


In [ ]:
trainData = pd.read_csv("train.csv")
testData = pd.read_csv("test.csv")
display(trainData.head(5))
display(testData.head(5))

In [ ]:
trainData = trainData.drop(['Id','Prediction'], 1)

In [ ]:
'''
Visualization of the data in order to have a first insight of the features and the distribution of non computable("NC")
values i.e -999.000
'''
NCFeatures = []
NCValues = []
for i in range(trainData.shape[1]):

        b = np.where(features_train[:,i] <= -999.000)[0]
        if (len(b) >= 1000):
            
            NCFeatures.append(i)
            NCValues.append(len(b))
            b = 0
            
        print('Feature ' + str(i) + ' Histogram')
        plt.hist(trainData.iloc[:,i])
        plt.show()         
                     

In [ ]:
df = pd.DataFrame(NCValues, index = NCFeatures)
display(df)

# Ivan's Blocks

In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from helpers import *
from proj1_helpers import * 
from play_with_data import * # Contains some functions that allow us to look the data and manipulate a little bit
from pre_processing import * # Contains all the functions needed to do some pre-processing on the data
%load_ext autoreload
%autoreload 2

In [2]:
# ********** Import csv file ***********

# Import of train.csv --> This was my way of doing it
# id_train, labels_train, features_train = load_data("train.csv")

# By using the helpers 
data_path = "train.csv"
labels_train, features_train, id_train = load_csv_data(data_path)

# Import of test.csv --> This is my way of doing it, because it is not done in proj1_helpers
id_test, features_test = load_data("test.csv")

In [3]:
# ********* Data whitening and "build model" ***********
# I think the order of the two functions below is important, because if we apply the whitening on the column of 1 it doesn't make sense I thin, do you???????

features_train = data_whitening(features_train) # to whiten the data
# Create the data for the model --> add a column of "1" as the first column --> this is for the w0 (offset parameter)
features_train = build_model_data(features_train, labels_train)[1] # the "[1]" is to take only the "X" matrix and not the "y"

In [4]:
features_bad_ind = np.unique(np.where(features_train == -999.0)[1])
features_train = np.delete(features_train,features_bad_ind,1)
print(features_train.shape)

(250000, 20)


In [5]:
for i in range(features_train.shape[1]):
    for j in list(range(i + 1)):    
        newFeature = np.multiply(features_train[:,i],features_train[:,j])
        features_train = np.c_[features_train,newFeature]


In [6]:
y=labels_train[0:1000]
tx=features_train[0:1000,:]
initial_w=np.zeros((tx.shape[1], 1))
max_iter=1000
gamma=0.7
lambda_=0.5



In [7]:
from logistic_regression import *
learning_grad_descent(y, tx, initial_w, max_iter, gamma)


iteration=0
loss (without penalization)=[ 693.14718056]
iteration=1
loss (without penalization)=[-6012220.29731315]
iteration=2
loss (without penalization)=[-8912556.56603811]
iteration=3
loss (without penalization)=[-11812892.83476306]
iteration=4
loss (without penalization)=[-14713229.10348802]
iteration=5
loss (without penalization)=[-17613565.37221298]
iteration=6
loss (without penalization)=[-20513901.64093794]
iteration=7
loss (without penalization)=[-23414237.90966289]
iteration=8
loss (without penalization)=[-26314574.17838785]
iteration=9
loss (without penalization)=[-29214910.44711281]
iteration=10
loss (without penalization)=[-32115246.71583777]
iteration=11
loss (without penalization)=[-35015582.98456272]
iteration=12
loss (without penalization)=[-37915919.25328768]
iteration=13
loss (without penalization)=[-40816255.52201264]
iteration=14
loss (without penalization)=[-43716591.79073759]
iteration=15
loss (without penalization)=[-46616928.05946255]
iteration=16
loss (withou

C:\Users\Ivan\Desktop\Etudes\GitHub\ML_project1\Notebook\logistic_regression.py:14: RuntimeWarning: overflow encountered in exp
  return 1.0 / (np.exp(-x) + 1)



loss (without penalization)=[ -1.04623653e+08]
iteration=36
loss (without penalization)=[ -1.07523990e+08]
iteration=37
loss (without penalization)=[ -1.10424326e+08]
iteration=38
loss (without penalization)=[ -1.13324662e+08]
iteration=39
loss (without penalization)=[ -1.16224999e+08]
iteration=40
loss (without penalization)=[ -1.19125335e+08]
iteration=41
loss (without penalization)=[ -1.22025671e+08]
iteration=42
loss (without penalization)=[ -1.24926007e+08]
iteration=43
loss (without penalization)=[ -1.27826344e+08]
iteration=44
loss (without penalization)=[ -1.30726680e+08]
iteration=45
loss (without penalization)=[ -1.33627016e+08]
iteration=46
loss (without penalization)=[ -1.36527352e+08]
iteration=47
loss (without penalization)=[ -1.39427689e+08]
iteration=48
loss (without penalization)=[ -1.42328025e+08]
iteration=49
loss (without penalization)=[ -1.45228361e+08]
iteration=50
loss (without penalization)=[ -1.48128697e+08]
iteration=51
loss (without penalization)=[ -1.510290

iteration=176
loss (without penalization)=[ -5.13571067e+08]
iteration=177
loss (without penalization)=[ -5.16471404e+08]
iteration=178
loss (without penalization)=[ -5.19371740e+08]
iteration=179
loss (without penalization)=[ -5.22272076e+08]
iteration=180
loss (without penalization)=[ -5.25172412e+08]
iteration=181
loss (without penalization)=[ -5.28072749e+08]
iteration=182
loss (without penalization)=[ -5.30973085e+08]
iteration=183
loss (without penalization)=[ -5.33873421e+08]
iteration=184
loss (without penalization)=[ -5.36773757e+08]
iteration=185
loss (without penalization)=[ -5.39674094e+08]
iteration=186
loss (without penalization)=[ -5.42574430e+08]
iteration=187
loss (without penalization)=[ -5.45474766e+08]
iteration=188
loss (without penalization)=[ -5.48375103e+08]
iteration=189
loss (without penalization)=[ -5.51275439e+08]
iteration=190
loss (without penalization)=[ -5.54175775e+08]
iteration=191
loss (without penalization)=[ -5.57076111e+08]
iteration=192
loss (with

iteration=319
loss (without penalization)=[ -9.28319154e+08]
iteration=320
loss (without penalization)=[ -9.31219490e+08]
iteration=321
loss (without penalization)=[ -9.34119826e+08]
iteration=322
loss (without penalization)=[ -9.37020163e+08]
iteration=323
loss (without penalization)=[ -9.39920499e+08]
iteration=324
loss (without penalization)=[ -9.42820835e+08]
iteration=325
loss (without penalization)=[ -9.45721171e+08]
iteration=326
loss (without penalization)=[ -9.48621508e+08]
iteration=327
loss (without penalization)=[ -9.51521844e+08]
iteration=328
loss (without penalization)=[ -9.54422180e+08]
iteration=329
loss (without penalization)=[ -9.57322516e+08]
iteration=330
loss (without penalization)=[ -9.60222853e+08]
iteration=331
loss (without penalization)=[ -9.63123189e+08]
iteration=332
loss (without penalization)=[ -9.66023525e+08]
iteration=333
loss (without penalization)=[ -9.68923862e+08]
iteration=334
loss (without penalization)=[ -9.71824198e+08]
iteration=335
loss (with

iteration=466
loss (without penalization)=[ -1.35466859e+09]
iteration=467
loss (without penalization)=[ -1.35756892e+09]
iteration=468
loss (without penalization)=[ -1.36046926e+09]
iteration=469
loss (without penalization)=[ -1.36336959e+09]
iteration=470
loss (without penalization)=[ -1.36626993e+09]
iteration=471
loss (without penalization)=[ -1.36917027e+09]
iteration=472
loss (without penalization)=[ -1.37207060e+09]
iteration=473
loss (without penalization)=[ -1.37497094e+09]
iteration=474
loss (without penalization)=[ -1.37787128e+09]
iteration=475
loss (without penalization)=[ -1.38077161e+09]
iteration=476
loss (without penalization)=[ -1.38367195e+09]
iteration=477
loss (without penalization)=[ -1.38657228e+09]
iteration=478
loss (without penalization)=[ -1.38947262e+09]
iteration=479
loss (without penalization)=[ -1.39237296e+09]
iteration=480
loss (without penalization)=[ -1.39527329e+09]
iteration=481
loss (without penalization)=[ -1.39817363e+09]
iteration=482
loss (with

iteration=613
loss (without penalization)=[ -1.78101802e+09]
iteration=614
loss (without penalization)=[ -1.78391835e+09]
iteration=615
loss (without penalization)=[ -1.78681869e+09]
iteration=616
loss (without penalization)=[ -1.78971903e+09]
iteration=617
loss (without penalization)=[ -1.79261936e+09]
iteration=618
loss (without penalization)=[ -1.79551970e+09]
iteration=619
loss (without penalization)=[ -1.79842003e+09]
iteration=620
loss (without penalization)=[ -1.80132037e+09]
iteration=621
loss (without penalization)=[ -1.80422071e+09]
iteration=622
loss (without penalization)=[ -1.80712104e+09]
iteration=623
loss (without penalization)=[ -1.81002138e+09]
iteration=624
loss (without penalization)=[ -1.81292172e+09]
iteration=625
loss (without penalization)=[ -1.81582205e+09]
iteration=626
loss (without penalization)=[ -1.81872239e+09]
iteration=627
loss (without penalization)=[ -1.82162272e+09]
iteration=628
loss (without penalization)=[ -1.82452306e+09]
iteration=629
loss (with

iteration=757
loss (without penalization)=[ -2.19866644e+09]
iteration=758
loss (without penalization)=[ -2.20156678e+09]
iteration=759
loss (without penalization)=[ -2.20446711e+09]
iteration=760
loss (without penalization)=[ -2.20736745e+09]
iteration=761
loss (without penalization)=[ -2.21026778e+09]
iteration=762
loss (without penalization)=[ -2.21316812e+09]
iteration=763
loss (without penalization)=[ -2.21606846e+09]
iteration=764
loss (without penalization)=[ -2.21896879e+09]
iteration=765
loss (without penalization)=[ -2.22186913e+09]
iteration=766
loss (without penalization)=[ -2.22476947e+09]
iteration=767
loss (without penalization)=[ -2.22766980e+09]
iteration=768
loss (without penalization)=[ -2.23057014e+09]
iteration=769
loss (without penalization)=[ -2.23347047e+09]
iteration=770
loss (without penalization)=[ -2.23637081e+09]
iteration=771
loss (without penalization)=[ -2.23927115e+09]
iteration=772
loss (without penalization)=[ -2.24217148e+09]
iteration=773
loss (with

iteration=903
loss (without penalization)=[ -2.62211553e+09]
iteration=904
loss (without penalization)=[ -2.62501587e+09]
iteration=905
loss (without penalization)=[ -2.62791621e+09]
iteration=906
loss (without penalization)=[ -2.63081654e+09]
iteration=907
loss (without penalization)=[ -2.63371688e+09]
iteration=908
loss (without penalization)=[ -2.63661722e+09]
iteration=909
loss (without penalization)=[ -2.63951755e+09]
iteration=910
loss (without penalization)=[ -2.64241789e+09]
iteration=911
loss (without penalization)=[ -2.64531822e+09]
iteration=912
loss (without penalization)=[ -2.64821856e+09]
iteration=913
loss (without penalization)=[ -2.65111890e+09]
iteration=914
loss (without penalization)=[ -2.65401923e+09]
iteration=915
loss (without penalization)=[ -2.65691957e+09]
iteration=916
loss (without penalization)=[ -2.65981991e+09]
iteration=917
loss (without penalization)=[ -2.66272024e+09]
iteration=918
loss (without penalization)=[ -2.66562058e+09]
iteration=919
loss (with

(array([[ -2.11750000e+05],
        [ -1.96716257e+05],
        [ -3.19749795e+03],
        [  1.36263007e+05],
        [  1.46374401e+02],
        [ -1.35863236e+04],
        [  1.04999148e+05],
        [ -1.20479691e+05],
        [  1.83508697e+05],
        [  1.59778436e+05],
        [ -2.71567546e+04],
        [  2.91764835e+03],
        [ -9.21370299e+03],
        [ -3.96826802e+04],
        [ -5.29558523e+03],
        [  4.24283723e+04],
        [  1.24598144e+03],
        [  9.32981456e+04],
        [  6.28177700e+04],
        [  8.94894643e+04],
        [ -2.11750000e+05],
        [ -1.96716257e+05],
        [ -2.07860703e+05],
        [ -3.19749795e+03],
        [ -6.42016736e+04],
        [ -4.66624992e+05],
        [  1.36263007e+05],
        [  7.18105559e+04],
        [  6.39141503e+03],
        [ -7.48461579e+03],
        [  1.46374401e+02],
        [ -3.47699854e+04],
        [ -3.14767678e+05],
        [  2.35028828e+03],
        [ -3.10048250e+05],
        [ -1.3586323